# Deep Learning - Houses Kaggle

In [12]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload
%autoreload 2

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn import set_config;set_config(display='diagram')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data = pd.read_csv("data/houses_kraggle.csv")
X = data.drop(columns='SalePrice')
y = data['SalePrice']

In [4]:
X.shape, y.shape

((1460, 80), (1460,))

## Preproc pipeline

In [7]:
def create_preproc_ordinal():
    feat_ordinal_dict = {
        # considers "missing" as the worse category
        "BsmtCond": ['missing', 'Po', 'Fa', 'TA', 'Gd'],
        "BsmtExposure": ['missing', 'No', 'Mn', 'Av', 'Gd'],
        "BsmtFinType1": ['missing', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
        "BsmtFinType2": ['missing', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
        "BsmtQual": ['missing', 'Fa', 'TA', 'Gd', 'Ex'],
        "Electrical": ['missing', 'Mix', 'FuseP', 'FuseF', 'FuseA', 'SBrkr'],
        "ExterCond": ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
        "ExterQual": ['missing', 'Fa', 'TA', 'Gd', 'Ex'],
        "Fence": ['missing', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv'],
        "FireplaceQu": ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
        "Functional": ['missing', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
        "GarageCond": ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
        "GarageFinish": ['missing', 'Unf', 'RFn', 'Fin'],
        "GarageQual": ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
        "HeatingQC": ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
        "KitchenQual": ['missing', 'Fa', 'TA', 'Gd', 'Ex'],
        "LandContour": ['missing', 'Low', 'Bnk', 'HLS', 'Lvl'],
        "LandSlope": ['missing', 'Sev', 'Mod', 'Gtl'],
        "LotShape": ['missing', 'IR3', 'IR2', 'IR1', 'Reg'],
        "PavedDrive": ['missing', 'N', 'P', 'Y'],
        "PoolQC": ['missing', 'Fa', 'Gd', 'Ex'],
    }

    feat_ordinal = sorted(feat_ordinal_dict.keys())
    feat_ordinal_values_sorted = [feat_ordinal_dict[i] for i in feat_ordinal]

    encoder_ordinal = OrdinalEncoder(
        categories=feat_ordinal_values_sorted,
        handle_unknown="use_encoded_value",
        unknown_value=-1  # Considers unknown values as worse than "missing"
    )

    preproc_ordinal = make_pipeline(
        SimpleImputer(strategy="constant", fill_value="missing"),
        encoder_ordinal,
        MinMaxScaler()
    )

    return preproc_ordinal, feat_ordinal


def create_preproc_numerical():
    return make_pipeline(
        KNNImputer(),
        MinMaxScaler()
    )

def create_preproc_nominal():
    return make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OneHotEncoder(handle_unknown="ignore")
    )

def create_preproc(X, percentile = 75):
    """Create an (unfitted) sklearn preprocessing pipeline well suited for the Houses Kaggle Competition

    Args:
        X (pandas.DataFrame): Feature matrix
        percentile (float): Percentage of feature to keep after one hot encoding

    Returns:
        sklearn.pipeline.Pipeline: un-fitted preprocessor
    """
    preproc_ordinal, feat_ordinal = create_preproc_ordinal()
    
    preproc_numerical = create_preproc_numerical()
    feat_numerical = sorted(X.select_dtypes(
        include=["int64", "float64"]).columns)
    
    preproc_nominal = create_preproc_nominal()
    feat_nominal = sorted(
        list(set(X.columns) - set(feat_numerical) - set(feat_ordinal)))
    
    feature_transformer = ColumnTransformer(
        [
        ("numerical_encoder", preproc_numerical, feat_numerical),
        ("ordinal_encoder", preproc_ordinal, feat_ordinal),
        ("nominal_encoder", preproc_nominal, feat_nominal)
        ],
        remainder="drop")

    feature_selector = SelectPercentile(
        mutual_info_regression,
        percentile=percentile,  # keep the xx% most important features
    )

    preproc = make_pipeline(
        feature_transformer,
        feature_selector
    )
    return preproc

In [8]:
preproc = create_preproc(X)
preproc

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical_encoder',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['1stFlrSF', '2ndFlrSF',
                                                   '3SsnPorch', 'BedroomAbvGr',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'BsmtUnfSF',
                                                   'EnclosedPorch',
                                                   'Fireplaces', 'FullBath',
                                                   'GarageArea', 'GarageCars...
                                                   'CentralAir', 'Condition1',
                                                   'Condition2', 'Exterior1st',
                                                   'Exterior2nd', 'Foundation',
                                                   'GarageType', 'Heating',
                                                   'HouseStyle', 'LotConfig',
                                                   'MSZoning', 'MasVnrType',
                                                   'MiscFeature',
                                                   'Neighborhood', 'RoofMatl',
                                                   'RoofStyle', 'SaleCondition',
                                                   'SaleType', 'Street',
                                                   'Utilities'])])),
                ('selectpercentile',
                 SelectPercentile(percentile=75,
                                  score_func=<function mutual_info_regression at 0x7f55d547bd30>))])

In [9]:
preproc.fit(X, y)
X_preproc = preproc.transform(X)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_preproc, y, test_size=0.3)

In [18]:
def initialize_model():

    model = models.Sequential()

    model.add(layers.Dense(20, activation='relu', input_dim=162)) # First hidden layer with 5 neurons
    model.add(layers.Dense(20, activation='relu')) # First hidden layer with 5 neurons
    model.add(layers.Dense(20, activation='relu')) # First hidden layer with 5 neurons

    #     model.add(layers.Dense(2, activation='relu')) # Third hidden layer with 3 neurons

    # Output layer that outputs a probability which is
    # necessary in the case of a 2 class classification problem
    model.add(layers.Dense(1, activation='linear')) 
    
    
    # Recommended compilator
    model.compile(optimizer='adam',
                  loss='msle')# directly optimize for the squared log error!
    return model

model = initialize_model()
history = model.fit(X_preproc, y,
                    epochs=100,
                    validation_split=0.3, # Play with this until your validation loss overfit
                    batch_size=16, # Keep batch size to 16 today
                    verbose=1)

Epoch 1/100
64/64 [==============================] - 0s 2ms/step - loss: 103.1384 - val_loss: 78.7289
Epoch 2/100
64/64 [==============================] - 0s 1ms/step - loss: 65.0859 - val_loss: 53.7477
Epoch 3/100
64/64 [==============================] - 0s 1ms/step - loss: 46.9093 - val_loss: 40.8507
Epoch 4/100
64/64 [==============================] - 0s 1ms/step - loss: 36.9319 - val_loss: 33.1980
Epoch 5/100
64/64 [==============================] - 0s 1ms/step - loss: 30.5968 - val_loss: 27.9778
Epoch 6/100
64/64 [==============================] - 0s 1ms/step - loss: 26.0869 - val_loss: 24.1080
Epoch 7/100
64/64 [==============================] - 0s 1ms/step - loss: 22.6531 - val_loss: 21.0826
Epoch 8/100
64/64 [==============================] - 0s 1ms/step - loss: 19.9244 - val_loss: 18.6332
Epoch 9/100
64/64 [==============================] - 0s 1ms/step - loss: 17.6895 - val_loss: 16.6091
Epoch 10/100
64/64 [==============================] - 0s 1ms/step - loss: 15.8210 - val_lo

64/64 [==============================] - 0s 985us/step - loss: 0.1034 - val_loss: 0.0942
Epoch 83/100
64/64 [==============================] - 0s 1ms/step - loss: 0.1021 - val_loss: 0.0931
Epoch 84/100
64/64 [==============================] - 0s 1ms/step - loss: 0.1009 - val_loss: 0.0921
Epoch 85/100
64/64 [==============================] - 0s 1ms/step - loss: 0.0999 - val_loss: 0.0913
Epoch 86/100
64/64 [==============================] - 0s 1ms/step - loss: 0.0990 - val_loss: 0.0905
Epoch 87/100
64/64 [==============================] - 0s 1ms/step - loss: 0.0982 - val_loss: 0.0899
Epoch 88/100
64/64 [==============================] - 0s 1ms/step - loss: 0.0974 - val_loss: 0.0894
Epoch 89/100
64/64 [==============================] - 0s 1ms/step - loss: 0.0968 - val_loss: 0.0888
Epoch 90/100
64/64 [==============================] - 0s 1ms/step - loss: 0.0962 - val_loss: 0.0883
Epoch 91/100
64/64 [==============================] - 0s 1ms/step - loss: 0.0957 - val_loss: 0.0879
Epoch 92/10

In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 20)                3260      
_________________________________________________________________
dense_9 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_10 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 21        
Total params: 4,121
Trainable params: 4,121
Non-trainable params: 0
_________________________________________________________________
